In [1]:
import numpy as np
import lightgbm as lgbm

In [2]:
'''
File paths
'''
import os

folder = os.path.join("/projectnb","cs542sp","netflix_wrw2", "CS542-final-project", "data")

In [3]:
next(os.walk(folder))[2]

['data_04.bin',
 'data_05.bin',
 'data_03.bin',
 'full_data.bin',
 'data_09.bin',
 'data_02.bin',
 'data_06.bin',
 'data_07.bin',
 'Baseline Info.docx',
 'data_01.bin',
 'data_08.bin',
 'data_00.bin',
 'who_rated_what_2006.sas7bdat',
 'netflix_analysis_dataset.sas7bdat',
 'who_rated_what_2006_ans_use.sas7bdat']

In [19]:
%%time

# read the lightgbm data for chunk 1
train = lgbm.Dataset(os.path.join(folder, 'data_03.bin'))

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 18.8 µs


In [20]:
# create the validation set
valid = lgbm.Dataset(os.path.join(folder, 'data_02.bin'), reference=train)

In [24]:
train.construct()
valid.construct()

[LightGBM] [Info] Load from binary file /projectnb/cs542sp/netflix_wrw2/CS542-final-project/data/data_03.bin


In [28]:
train.num_feature()

12

In [21]:
%%time

# build the model

'''
https://sites.google.com/view/lauraepp/parameters

learning_rate: < 0.05
early_stopping_round: 10% num rounds
max_depth: [3 12] ~ larger better but overfitting speed increase
max_leaves: 2^max_depth-1 
'''

params = {}
# binary classification
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'

# model constants (don't change these!)
params['learning_rate'] = 0.005
params['boosting_type'] = 'gbdt'

# tree size
params['max_depth'] = 7
params['num_leaves'] = 127

# speed up training rate with bagging
params['bagging_freq'] = 5
params['bagging_fraction'] = 0.5

num_rounds = 100

# early stopping rounds requires validation set!

bst = lgbm.train(params, data, num_rounds, valid_sets = [valid], early_stopping_rounds=10)

[LightGBM] [Info] Load from binary file /projectnb/cs542sp/netflix_wrw2/CS542-final-project/data/data_03.bin
[LightGBM] [Info] Number of positive: 8242599, number of negative: 11757154
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.862736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2154
[LightGBM] [Info] Number of data points in the train set: 19999753, number of used features: 12


LightGBMError: Cannot add validation data, since it has different bin mappers with training data

In [ ]:
# save the model
bst.save_model(os.path.join('models', 'model.txt'))